In [1]:
def Lane(binary_warped,Minv,original_image):
   nonzero = binary_warped.nonzero()  # Identify the x and y positions of all nonzero pixels in the image
   nonzeroy = np.array(nonzero[0])
   nonzerox = np.array(nonzero[1])
    
   # Create an output image to draw on and  visualize the result
   out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
         
   margin = 50   
   # Set minimum number of pixels found to recenter window   
   minpix = 50
    # Take a histogram of the bottom half of the image
   histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    # Find the peak of the left and right halves of the histogram
            # These will be the starting point for the left and right lines
   midpoint = np.int(histogram.shape[0]/2)
   leftx_base = np.argmax(histogram[:midpoint])
   rightx_base = np.argmax(histogram[midpoint:]) + midpoint

   nwindows = 9  # Choose the number of sliding windows
   window_height = np.int((binary_warped.shape[0])/nwindows)   # Set height of windows

   leftx_current = leftx_base   # Current positions to be updated for each window
   rightx_current = rightx_base   # Set the width of the windows +/- margin
   left_lane_inds = []
   right_lane_inds = []

            # Step through the windows one by one
   for window in range(nwindows):

                # Identify window boundaries in x and y (and right and left)
       win_y_low = binary_warped.shape[0] - (window+1)*window_height
       win_y_high = binary_warped.shape[0] - window*window_height
       win_xleft_low = leftx_current - margin
       win_xleft_high = leftx_current + margin
       win_xright_low = rightx_current - margin
       win_xright_high = rightx_current + margin

                # Identify the nonzero pixels in x and y within the window
       good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                                  (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
       good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                                   (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
            
                # Append these indices to the lists
       left_lane_inds.append(good_left_inds)
       right_lane_inds.append(good_right_inds)
                # If you found > minpix pixels, recenter next window on their mean position
       if len(good_left_inds) > minpix:
                    leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
       if len(good_right_inds) > minpix:        
                    rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

   
     # Concatenate the arrays of indices
   left_lane_inds = np.concatenate(left_lane_inds)
   right_lane_inds = np.concatenate(right_lane_inds)
     
        # Again, extract left and right line pixel positions
   leftx = nonzerox[left_lane_inds]
   lefty = nonzeroy[left_lane_inds] 
   rightx = nonzerox[right_lane_inds]
   righty = nonzeroy[right_lane_inds]
        
   ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
   if lefty.shape[0] >= 400 and righty.shape[0] >= 400 and leftx.shape[0] >= 400 and rightx.shape[0] >= 400:
           self.detected = False
   left_fit = np.polyfit(lefty, leftx, 2)
   right_fit = np.polyfit(righty, rightx, 2)
   
            # Generate x and y values for plotting
   left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
   right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty +right_fit[2]
   warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
   color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

   pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
   pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
   pts = np.hstack((pts_left, pts_right))
   cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
   newwarp = cv2.warpPerspective(color_warp, Minv, (original_image.shape[1], original_image.shape[0])) 
   result = cv2.addWeighted(original_image, 1, newwarp, 0.3, 0)
  
   
   return result